### Reading all PDFs in a directory and storing their content in a DataFrame with a progress bar

- items downloaded from loc.gov 
- based on excel spreadsheet of cases from penn state supreme court database (https://scdb.la.psu.edu/) 1946-2024
  - consider the other dbs provided with issueArea, issues, direction, etc 
  - https://scdb.la.psu.edu/online-codebook/decision-direction/

In [5]:
import pandas as pd

## read all the PDFS into a dataframe from ../scotus_pdfs
import os
from PyPDF2 import PdfReader
import pandas as pd
import re

df = pd.DataFrame(columns=["filename", "content"])

def read_pdf(file_path):
    with open(file_path, "rb") as f:
        reader = PdfReader(f)
        text = ""
        for page in reader.pages:
            text += page.extract_text()
        
        # Fix hyphenated line breaks (e.g., "ordinary mean-\ning" -> "ordinary meaning")
        text = re.sub(r'-\n', '', text)
        
        # Clean up other common PDF extraction issues
        text = re.sub(r'\n+', ' ', text)  # Replace multiple newlines with single space
        text = re.sub(r'\s+', ' ', text)  # Replace multiple spaces with single space
        
        # Additional cleaning for legal documents
        text = re.sub(r'\f', ' ', text)  # Remove form feeds
        text = re.sub(r'[^\w\s\.\,\;\:\!\?\-\(\)\[\]\"\'\/]', ' ', text)  # Keep only common punctuation
        
        text = text.strip()
        
        return text

In [6]:
from tqdm import tqdm

for filename in tqdm(os.listdir("../scotus_pdfs")):
    if filename.endswith(".pdf"):
        #print(f"Reading {filename}")
        file_path = os.path.join("../scotus_pdfs", filename)
        content = read_pdf(file_path)
        df = pd.concat([df, pd.DataFrame([{"filename": filename, "content": content}])], ignore_index=True)


 10%|█         | 852/8353 [02:31<22:10,  5.64it/s]


KeyboardInterrupt: 

In [ ]:
total_tokens = df["content"].apply(lambda x: len(x.split())).sum()
print("Total tokens (approximate by word count):", total_tokens)


Total tokens (approximate by word count): 63175725
